<h1>Colab Python Notebook For How Banks Can Save The World Pt 2</h1>

<h2>Applies Electra NLP model to company reports to identify projects funded by this organisation</h2>

Downloads tensorflow based NLP model 'electra' fine tuned for Q&A

Uses excerpts from company reports as context - question panel designed to pick the names of projects and initiatives that the company is funding.

The model produces an nbest file (not shown in this notebook - an exceprt will be uploaded to git hub) which contains any projects and initiatives that the company may be funding. 

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow.compat.v1 as tf
import tensorflow.compat.v1 as tensorflow
print(tensorflow.__version__)

1.15.2


In [ ]:
!git clone https://github.com/infospark/electra.git


Cloning into 'electra'...
remote: Enumerating objects: 378, done.
remote: Total 378 (delta 0), reused 0 (delta 0), pack-reused 378
Receiving objects: 100% (378/378), 11.88 MiB | 12.95 MiB/s, done.
Resolving deltas: 100% (249/249), done.


In [ ]:
cd /content/electra

/content/electra


# Set up Tensor Environment

In [ ]:
from google.colab import auth
auth.authenticate_user()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf


assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is => ', TPU_ADDRESS)


with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is =>  grpc://10.45.224.178:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 14538302335085740752),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 18121934246772018943),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 3350729021359451590),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 5052307395710909598),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 4067832406828410456),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 4716350520594257938),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 14102687750644912418),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 2291278774728361259),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 12048019

In [ ]:
%%writefile dev.json
{
    "version": "v2.0",
    "data": [
        {
            "title": "Project Identification",
            "paragraphs": [
                {
                    "qas": [
                        {
                            "question": "What initiative is in progress?",
                            "id": "q_initiative",
                            "answers": [],
                            "is_impossible": false
                        },{
                            "question": "What project is being funded?",
                            "id": "q_funded",
                            "answers": [],
                            "is_impossible": false
                        },{
                            "question": "What system is being evaluated?",
                            "id": "q_evaluated",
                            "answers": [],
                            "is_impossible": false
                        },{
                            "question": "What new products are being developed?",
                            "id": "q_new_products",
                            "answers": [],
                            "is_impossible": false
                        }
                    ],
                    "context": "LafargeHolcim’s wide range of low-carbon products (Ductal®, Chronolia®, Agilia®, Artevia®, or Thermedia 6B® for example) enables to reduce both CO2 emissions generated by the production process and CO2 emissions for our clients in the building sector. Furthermore, we are developing with our fundamental research centre new products with higher CO2 savings potential (eg. Aether (30% lower in CO2) or Solidia with a potential up to 70% CO2 reduction). Please note the 11 % revenue refers to the total percentage of sustainable Solutions in 2018. "

                }
            ]
        }
    ]
}

Writing dev.json


In [ ]:
!gsutil mv dev.json gs://jims_bert_test/electra_data_large/finetuning_data/squad/

Copying file://dev.json [Content-Type=application/json]...
Removing file://dev.json...

Operation completed over 1 objects/1.8 KiB.                                      


In [ ]:
!git pull
!gsutil rm gs://jims_bert_test/electra_data_large/models/electra_large/finetuning_tfrecords/squad_tfrecords/squad_dev.*
!python3 run_finetuning.py --data-dir gs://jims_bert_test/electra_data_large/ --model-name electra_large --hparams '{"tpu_name": "grpc://10.45.224.178:8470", "do_train": false, "do_eval": true, "model_size": "large", "task_names": ["squad"], "init_checkpoint": "gs://jims_bert_test/electra_data_large/models/electra_large/finetuning_models/squad_model_1", "use_tpu": "True", "num_tpu_cores":8}'
#!python3 run_finetuning.py --data-dir gs://jims_bert_test/electra_data/ --model-name electra_base --hparams '{"do_train": false, "do_eval": true, "model_size": "base", "task_names": ["squad"], "init_checkpoint": "gs://jims_bert_test/electra_data/models/electra_base/finetuning_models/squad_model_1", "use_tpu": "False"}'

Already up to date.
Removing gs://jims_bert_test/electra_data_large/models/electra_large/finetuning_tfrecords/squad_tfrecords/squad_dev.metadata...
Removing gs://jims_bert_test/electra_data_large/models/electra_large/finetuning_tfrecords/squad_tfrecords/squad_dev.tfrecord...
/ [2 objects]                                                                   
Operation completed over 2 objects.                                              
Config: model=electra_large, trial 1/1
answerable_classifier True
answerable_uses_start_logits True
answerable_weight 0.5
beam_size 20
data_dir gs://jims_bert_test/electra_data_large/
debug False
do_eval True
do_lower_case True
do_train False
doc_stride 128
double_unordered True
embedding_size None
eval_batch_size 32
exact_file <built-in method format of str object at 0x7ff7e93e98a0>
f1_file <built-in method format of str object at 0x7ff7e93e9930>
gcp_project None
init_checkpoint gs://jims_bert_test/electra_data_large/models/electra_large/finetuning_model

In [ ]:
!gsutil cat gs://jims_bert_test/electra_data_large/models/electra_large/results/squad_qa/squad_null_odds.json

{"q_initiative": -2.0232818126678467, "q_funded": 6.843087196350098, "q_evaluated": 4.941781997680664, "q_new_products": -4.185152053833008}

In [ ]:
# Sample output from nbest file - the top N answers for each question:
"q_evaluated": [
    {
      "text": "Aether",
      "probability": [
        0
      ],
      "start_logit": -6.888813495635986,
      "end_logit": -0.6815726161003113,
      "start_index": 53,
      "end_index": 53
    },
    {
      "text": "Aether (30% lower in CO2) or Solidia",
      "probability": [
        1
      ],
      "start_logit": -6.888813495635986,
      "end_logit": -0.8862883448600769,
      "start_index": 53,
      "end_index": 59
    },
    {
      "text": "Solidia",
      "probability": [
        2
      ],
      "start_logit": -7.253624439239502,
      "end_logit": -0.7757238745689392,
      "start_index": 59,
      "end_index": 59
    },